In [1]:
import pandas as pd
import dask.dataframe as dd
from dask.utils import tmpfile
from sqlalchemy import create_engine, text
import sqlite3

In [2]:
ddf = dd.read_json('yelp_academic_dataset_business.json', lines=True, blocksize=3e6)
ddf = ddf.drop(['attributes', 'hours'], axis=1)

In [10]:
type(ddf)
#ddf.describe().compute()

dask.dataframe.core.DataFrame

In [9]:
len(ddf)

150346

In [26]:
ddf['state'].compute().value_counts()

PA     34039
FL     26330
TN     12056
IN     11247
MO     10913
LA      9924
AZ      9912
NJ      8536
NV      7715
AB      5573
CA      5203
ID      4467
DE      2265
IL      2145
TX         4
CO         3
MA         2
WA         2
HI         2
SD         1
XMS        1
UT         1
VT         1
NC         1
MI         1
VI         1
MT         1
Name: state, dtype: int64

In [28]:
cities = ddf[['city', 'state']].compute()

In [31]:
cacities = cities[cities['state'] == 'CA']

In [35]:
cacities['city'].value_counts()

Santa Barbara                       3829
Goleta                               798
Carpinteria                          298
Isla Vista                            94
Montecito                             93
Summerland                            41
Truckee                               11
Santa Barbara                          5
Reno                                   3
Santa Barbra                           2
Salinas                                1
Santa Maria                            1
Kings Beach                            1
SANTA BARBARA                          1
SANTA BARBARA AP                       1
Ventura                                1
Santa Ynez                             1
South Lake Tahoe                       1
Tampa                                  1
Santa Clara                            1
Meridian                               1
Port Hueneme                           1
Santa  Barbara                         1
Oxnard                                 1
Los Angeles     

In [38]:
ddf_reviews = dd.read_json('yelp_academic_dataset_review.json', lines=True, blocksize=3e6)

In [39]:
len(ddf_reviews)

6990280

In [40]:
ddf_reviews.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15


In [43]:
ddf.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,"Doctors, Traditional Chinese Medicine, Naturop..."
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,"Shipping Centers, Local Services, Notaries, Ma..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"Department Stores, Shopping, Fashion, Home & G..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"Restaurants, Food, Bubble Tea, Coffee & Tea, B..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"Brewpubs, Breweries, Food"


In [44]:
ddf = ddf[ddf['categories'].str.contains('Restaurants', na=False)]
pa_rest = ddf[ddf['state'].str.contains('PA')]
pa_rest.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"Restaurants, Food, Bubble Tea, Coffee & Tea, B..."
15,MUTTqe8uqyMdBl186RmNeA,Tuna Bar,205 Race St,Philadelphia,PA,19106,39.953949,-75.143226,4.0,245,1,"Sushi Bars, Restaurants, Japanese"
19,ROeacJQwBeh05Rqg7F6TCg,BAP,1224 South St,Philadelphia,PA,19147,39.943223,-75.162568,4.5,205,1,"Korean, Restaurants"
28,QdN72BWoyFypdGJhhI5r7g,Bar One,767 S 9th St,Philadelphia,PA,19147,39.939825,-75.157447,4.0,65,0,"Cocktail Bars, Bars, Italian, Nightlife, Resta..."
31,Mjboz24M9NlBeiOJKLEd_Q,DeSandro on Main,4105 Main St,Philadelphia,PA,19127,40.022466,-75.218314,3.0,41,0,"Pizza, Restaurants, Salad, Soup"


In [45]:
len(pa_rest)

12641

In [46]:
type(pa_rest)

dask.dataframe.core.DataFrame

In [48]:
pa_rest_reviews = pa_rest.merge(ddf_reviews, on='business_id', how='left').compute()
pa_rest_reviews

ignoring exception in ensure_cleanup_on_exception
Traceback (most recent call last):
  File "/home/william/anaconda3/lib/python3.7/site-packages/dask/dataframe/shuffle.py", line 875, in ensure_cleanup_on_exception
    yield
  File "/home/william/anaconda3/lib/python3.7/site-packages/dask/dataframe/shuffle.py", line 890, in shuffle_group_3
    p.append(d, fsync=True)
  File "/home/william/anaconda3/lib/python3.7/site-packages/partd/encode.py", line 25, in append
    self.partd.append(data, **kwargs)
  File "/home/william/anaconda3/lib/python3.7/site-packages/partd/buffer.py", line 45, in append
    self.flush(keys)
  File "/home/william/anaconda3/lib/python3.7/site-packages/partd/buffer.py", line 99, in flush
    self.slow.append(dict(zip(keys, self.fast.get(keys))))
  File "/home/william/anaconda3/lib/python3.7/site-packages/partd/file.py", line 46, in append
    os.fsync(f)
OSError: [Errno 28] No space left on device

During handling of the above exception, another exception occurred:

OSError: [Errno 28] No space left on device

In [36]:
ddf_user = dd.read_json('yelp_academic_dataset_user.json', lines=True, blocksize=3e6)

In [37]:
ddf_user.head()

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,...,65,55,56,18,232,844,467,467,239,180
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,...,264,184,157,251,1847,7054,3131,3131,1521,1946
2,2WnXYQFK0hXEoTxPtV2zvg,Steph,665,2008-07-25 10:41:00,2086,1010,1003,"2009,2010,2011,2012,2013","LuO3Bn4f3rlhyHIaNfTlnA, j9B4XdHUhDfTKVecyWQgyA...",52,...,13,10,17,3,66,96,119,119,35,18
3,SZDeASXq7o05mMNLshsdIA,Gwen,224,2005-11-29 04:38:33,512,330,299,"2009,2010,2011","enx1vVPnfdNUdPho6PH_wg, 4wOcvMLtU6a9Lslggq74Vg...",28,...,4,1,6,2,12,16,26,26,10,9
4,hA5lMy-EnncsH4JoR-hFGQ,Karen,79,2007-01-05 19:40:59,29,15,7,,"PBK4q9KEEBHhFvSXCUirIw, 3FWPpM7KU1gXeOM_ZbYMbA...",1,...,1,0,0,0,1,1,0,0,0,0


In [41]:
pacities = cities[cities['state'] == 'PA']
pacities['city'].value_counts()

Philadelphia        14567
West Chester          838
King of Prussia       560
Doylestown            539
Bensalem              454
                    ...  
Devon-Berwyn            1
frazer                  1
West Chester PA         1
kop                     1
MIDDLE CITY WEST        1
Name: city, Length: 434, dtype: int64

In [42]:
pacities

,city,state
3,Philadelphia,PA
4,Green Lane,PA
15,Philadelphia,PA
19,Philadelphia,PA
25,Newtown,PA
...,...,...
2331,Clifton Heights,PA
2332,Philadelphia,PA
2335,Philadelphia,PA
2340,Philadelphia,PA


In [ ]:
db = 'sqlite:///your_database.db'
ddf.to_sql('business', db, if_exists='replace')
engine = create_engine(db, echo=False)
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM business")).fetchall()

In [ ]:
# Commit and close the connection
#conn.commit()
conn.close()

In [ ]:
conn = sqlite3.connect('your_database.db')

In [ ]:
cursor = conn.cursor()

In [ ]:
df = pd.read_sql_query("SELECT * FROM data LIMIT 10", conn)

In [ ]:
df.head()

In [ ]:
tip_df = pd.read_sql_query("SELECT * FROM test LIMIT 10", conn)

In [ ]:
tip_df.head()

In [ ]:
review_ddf = dd.read_json('review_small.json', lines=True, blocksize=1e6)
review_ddf.to_sql('reviews', db)

In [ ]:
review_ddf.head()

In [ ]:
user_review_counts = review_ddf.groupby("user_id").size().compute()

In [ ]:
user_review_counts[user_review_counts > 4]


In [ ]:
reviews_all_ddf = dd.read_json('yelp_academic_dataset_review.json', lines=True, blocksize=1e6)